In [ ]:
# # MOUNTING GOOGLE DRIVE
# from google.colab import drive
# drive.mount('/content/drive')

# import os
# print(os.getcwd())

# wd = '/content/drive/MyDrive/CS 685/cs685_project/notebooks'
# print(os.listdir(wd))
# os.chdir(wd)
# print(os.getcwd())

In [ ]:
# !pip install tokenizers
# !pip install transformers
# !pip install sentencepiece

In [1]:
import os
import json
import numpy as np
from tqdm import trange

from load_dataset import Text2SQLDataset
from torch.utils.data import DataLoader

import torch
import torch.nn as nn
import torch.optim as optim
from tokenizers import AddedToken
from torch.utils.tensorboard import SummaryWriter
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import GPT2Tokenizer, GPT2Model, GPT2Config


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [2]:
# configuration = GPT2Config()
# configuration

## GPT2 + T5 network

In [4]:
# FOR PRINTING INTERMEDIATE TORCH SIZES
DEBUG_FLAG = False

In [5]:
# Define model
class EncoderDecoder(nn.Module):
    def __init__(self, gpt2_hidden_size, t5_hidden_size, max_input_length, 
                 max_output_length, gpt2_model, t5_model, batch_size, gpt2_tokenizer):
        super(EncoderDecoder, self).__init__()
        
        self.max_input_length = max_input_length
        self.max_output_length = max_output_length
        self.t5_hidden_size = t5_hidden_size
        self.batch_size = batch_size
        
        self.gpt2 = GPT2Model.from_pretrained(gpt2_model)
        self.gpt2.resize_token_embeddings(len(gpt2_tokenizer))
        
        self.t5 = T5ForConditionalGeneration.from_pretrained(t5_model)
        self.linear = nn.Linear(gpt2_hidden_size, t5_hidden_size)
    
        self.t5.config.is_encoder_decoder = False
    

    def forward(self, input_ids, input_mask,
                decoder_input_ids, decoder_attention_mask):
        
        # Encode input with GPT2
        gpt2_output = self.gpt2(input_ids=input_ids, attention_mask=input_mask)
        gpt2_output = gpt2_output.last_hidden_state
        
        if DEBUG_FLAG: print(f"gpt2_output - {gpt2_output.size()}")
        
        # Transform GPT2 output to T5 input shape
        t5_input = self.linear(gpt2_output)
        if DEBUG_FLAG: print(f"gpt2_output linear - {t5_input.size()}")
        
#         t5_input = t5_input.unsqueeze(1).repeat(1, self.max_output_length, 1)
#         if DEBUG_FLAG: print(f"gpt2_output linear unsqueeze - {t5_input.size()}")
        
#         t5_input = t5_input.view(self.batch_size, self.max_input_length, self.t5_hidden_size)
        t5_input = t5_input.unsqueeze(0)
        if DEBUG_FLAG: print(f"t5_input - {t5_input.size()}")
        
#         t5_outputs = self.t5(decoder_input_ids=decoder_input_ids,
#                              decoder_attention_mask=decoder_attention_mask,
#                              encoder_outputs=t5_input
#                             )
#         if DEBUG_FLAG: print(f"t5_input logits - {(t5_outputs.logits).size()}")
#         return t5_outputs.logits
    
        t5_outputs = self.t5(labels=decoder_input_ids,
                             decoder_attention_mask=decoder_attention_mask,
                             encoder_outputs=t5_input,
                             return_dict = True
                            )
        
        if DEBUG_FLAG: print(f"t5_input - {type(t5_outputs)}")
        return t5_outputs

    def predict(self, input_ids, input_mask, batch_size, t5_tokenizer):
        
        self.gpt2.eval()
        # Encode input with GPT2
        gpt2_output = self.gpt2(input_ids=input_ids, attention_mask=input_mask)
        gpt2_output = gpt2_output.last_hidden_state
        if DEBUG_FLAG: print(f"gpt2_output - {gpt2_output.size()}")
        
        # Transform BERT output to T5 input shape
        t5_input = self.linear(gpt2_output)
        if DEBUG_FLAG: print(f"gpt2_output linear - {t5_input.size()}")
            
#         t5_input = t5_input.view(batch_size, self.max_input_length, self.t5_hidden_size)
        t5_input = t5_input.unsqueeze(0)
        if DEBUG_FLAG: print(f"t5_input - {t5_input.size()} - {t5_input}")
        
        # Generate initial input for T5 decoder
        start_token = t5_tokenizer.pad_token_id
        
#         decoder_input_ids = torch.tensor([start_token] * batch_size).unsqueeze(0)
#         decoder_attention_mask = torch.tensor([1] * batch_size).unsqueeze(0)
        
        decoder_input_ids = torch.tensor([start_token]*batch_size).unsqueeze(0)
        decoder_attention_mask = torch.tensor([1]*batch_size).unsqueeze(0)
    
#         decoder_input_ids = decoder_input_ids.view(decoder_input_ids.shape[1],
#                                                    decoder_input_ids.shape[0])
#         decoder_attention_mask = decoder_attention_mask.view(decoder_attention_mask.shape[1],
#                                                              decoder_attention_mask.shape[0])
        
        print(f"decoder_input_ids - {decoder_input_ids.size()}")
        print(f"decoder_attention_mask - {decoder_attention_mask.size()}")
        
        print(f"initial decoder_input_ids - {decoder_input_ids}")
        
        # Use the model to get output logits
        # Predict the output
        self.t5.eval()
        with torch.no_grad():
            for i in range(50):  # Maximum length of generated sequence
                t5_outputs = self.t5(decoder_input_ids=decoder_input_ids,
                                     decoder_attention_mask=decoder_attention_mask,
                                     encoder_outputs=t5_input)
#                 print(f"t5_outputs - {t5_outputs}")
                print(f"t5_outputs logits - {(t5_outputs.logits).size()}")
    
                next_token_logits = t5_outputs.logits[:, -1, :]
                print(f"next_token_logits - {next_token_logits.size()}")
            
#                 next_token_id = torch.argmax(next_token_logits, dim=-1)
                next_token_id = next_token_logits.argmax(1)
#                 print(f"next_token_id - {next_token_id.size()}")
#                 print(f"next_token_id.unsqueeze(-1) - {next_token_id.unsqueeze(-1).size()}")
                decoder_input_ids = torch.cat([decoder_input_ids, next_token_id.unsqueeze(-1)], dim=-1)
                decoder_attention_mask = torch.cat([decoder_attention_mask,
                                                    torch.ones_like(next_token_id.unsqueeze(-1))], dim=-1)

                if next_token_id == t5_tokenizer.eos_token_id:
                    break
                
                print(f"pred decoder_input_ids - {decoder_input_ids}")
                
#                 break
        
        # generated_text
#         t5_outputs = t5_tokenizer.decode(decoder_input_ids.squeeze(), skip_special_tokens=True)
        t5_outputs = decoder_input_ids #.squeeze()
        
        return t5_outputs

In [8]:
def train(train_filepath, batch_size, gpt2_hidden_size, t5_hidden_size, lr, num_epochs,
         max_input_length, max_output_length, gpt2_model, t5_model):
    
    sub_folder_name = f"GPT2_T5_lr{lr}_bs{batch_size}_{gpt2_model}_{t5_model}"
    models_directory = f"models/{sub_folder_name}"

    if not os.path.isdir(models_directory):
        os.makedirs(models_directory)
        
    # TENSORBOARD
    writer = SummaryWriter(f'tb/loss_plot/{sub_folder_name}')
    
    train_dataset = Text2SQLDataset(
            dir_ = train_filepath,
            mode = "train")

    train_dataloder = DataLoader(
            train_dataset, 
            batch_size = batch_size, 
            shuffle = True,
            collate_fn = lambda x: x,
            drop_last = True
        )
    
    print(f"Number of batches - {len(train_dataloder)}")

    # Define GPT2 and T5 tokenizers
    gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    t5_tokenizer = T5Tokenizer.from_pretrained(t5_model)
    print(f"Tokenizers loaded")
    
    gpt2_tokenizer.add_special_tokens({'pad_token': '[PAD]'})

    model = EncoderDecoder(gpt2_hidden_size, t5_hidden_size, 
                           max_input_length, max_output_length,
                           gpt2_model, t5_model, batch_size, gpt2_tokenizer).to(device)
    print(f"Model loaded")
#     print(f"{model.config.decoder_start_token_id}")

    optimizer = optim.Adam(model.parameters(), lr=lr)
    print(f"Otimizer - Adam")

    criterion = nn.CrossEntropyLoss(ignore_index=t5_tokenizer.pad_token_id)
    print(f"CrossEntropyLoss initialized")
    
#     criterion = ContrastiveLoss()
    
    # initialize array of losses 
    losses = {'train': {}, "val": {}}

    # for epoch in range(num_epochs):
    with trange(num_epochs) as tr:
        for epoch in tr:
            
            # Train the model
            model.train()
            
            batch_loss = 0
            
            for idx, batch in enumerate(train_dataloder):

                batch_inputs = [data[0] for data in batch]
                batch_sqls = [data[1] for data in batch]

                if DEBUG_FLAG:
                    if epoch == 0 and idx == 0:
                        print(f"batch_inputs - {type(batch_inputs)} {len(batch_inputs)}")
                        print(f"batch_sqls - {type(batch_sqls)} {len(batch_sqls)}")
                        
#                 for temp_i, temp in enumerate(batch_inputs):
#                     print(f"batch_inputs - {batch_inputs[temp_i]}")
#                     print(f"batch_sqls - {batch_sqls[temp_i]}")

                tokenized_inputs = gpt2_tokenizer(batch_inputs,
                                                  add_special_tokens=True,
                                                  padding="max_length", #True,
                                                  max_length=max_input_length,
                                                  #pad_to_max_length=True,
                                                  return_tensors='pt',
                                                  truncation=True)

                encoder_input_ids = tokenized_inputs["input_ids"].to(device)
                encoder_input_attention_mask = tokenized_inputs["attention_mask"].to(device)

#                 print(f"encoder_input_ids - {encoder_input_ids}")
                tokenized_outputs = t5_tokenizer(batch_sqls,
                                                 add_special_tokens=True,
                                                 padding="max_length", #True,
                                                 max_length=max_output_length,
                                                 #pad_to_max_length=True,
                                                 return_tensors='pt',
                                                 truncation=True)


                decoder_input_ids = tokenized_outputs["input_ids"].to(device)
                # replace padding token id's of the labels by -100 so it's ignored by the loss
                decoder_input_ids[decoder_input_ids == t5_tokenizer.pad_token_id] = -100
                decoder_attention_mask = tokenized_outputs["attention_mask"].to(device)
#                 labels = None #tokenized_outputs["attention_mask"].to(device)

#                 print(f"decoder_input_ids - {decoder_input_ids}")

                if DEBUG_FLAG and epoch == 0 and idx == 0:
                    print(f"encoder_input_ids - {encoder_input_ids.size()}")
                    print(f"encoder_input_attention_mask - {encoder_input_attention_mask.size()}")
                    print(f"decoder_input_ids - {decoder_input_ids.size()}")
                    print(f"decoder_attention_mask - {decoder_attention_mask.size()}")

                # Clear gradients
                optimizer.zero_grad()

                model_output = model(encoder_input_ids,
                               encoder_input_attention_mask,
                               decoder_input_ids,
                               decoder_attention_mask)
#                                labels=labels)
                
                output = model_output["logits"]
#                 print(f"output - {output.size()}")
#                 print(f"decoder_input_ids - {decoder_input_ids.size()}")
                
                output_resize = output.view(output.shape[0]*output.shape[1], output.shape[2])
                decoder_input_ids_resize = decoder_input_ids.view(decoder_input_ids.shape[0]*decoder_input_ids.shape[1])
                
                print(f"output_resize - {output_resize.size()}")
                print(f"decoder_input_ids_resize - {decoder_input_ids_resize.size()}")
                    
#                 loss = criterion(output_resize, decoder_input_ids_resize)
#                 batch_loss += loss.item()
                
#                 print(f"output - {model_output}")
                loss = model_output["loss"]
                batch_loss += loss
                
                predicted_classes = torch.argmax(output_resize, dim=-1)
                print(f"output_resize - {predicted_classes.size} - {predicted_classes}")
                
#                 print(f"decoder_input_ids_resize - {decoder_input_ids_resize}")

                # backpropagation
                loss.backward()
                optimizer.step()

                break
                
            batch_loss /= len(train_dataloder) 
            losses['train'][epoch] = f"{batch_loss:.3f}"
            #progress bar 
            tr.set_postfix({"epoch_num":epoch,
                            "loss":f"{batch_loss:.10f}"})
            
#             with open(os.path.join(models_directory, "loss.json"), 'w') as f:
#                 json.dump(losses, f)
            
#             writer.add_scalar('Training loss', batch_loss, global_step=epoch+1)
#             # save models
#             if (epoch > 3 and epoch % 5 == 0):
#                 torch.save(model, os.path.join(models_directory, f"model_{epoch}"))
#     torch.save(model, os.path.join(models_directory, f"model_last_{epoch}"))


In [9]:
# Define hyperparameters

train(train_filepath = "../data/resdsql_pre/preprocessed_dataset_train.json",
      batch_size = 2, #32
      gpt2_hidden_size = 768,
      t5_hidden_size = 512,
      lr = 1e-4,
      num_epochs = 1, #300
      max_input_length = 43,
      max_output_length = 127,
      gpt2_model = 'gpt2',
      t5_model = 't5-small')

Number of batches - 3152
Tokenizers loaded
Model loaded
Otimizer - Adam
CrossEntropyLoss initialized


  0%|                                                                                                                                                                 | 0/1 [00:00<?, ?it/s]

output_resize - torch.Size([254, 32128])
decoder_input_ids_resize - torch.Size([254])


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.20s/it, epoch_num=0, loss=0.0042370888]


## Evaluation

In [35]:
# Model class must be defined somewhere

model_folder = "models/GPT2_T5_lr0.0001_bs32_gpt2_t5-small/"
model_name = "model_45"

model_path = os.path.join(os.getcwd(), model_folder, model_name)
output = os.path.join(os.getcwd(), model_folder, f"{model_name}.txt")

model2 = torch.load(model_path, map_location=torch.device('cpu'))
model2.eval()

EncoderDecoder(
  (gpt2): GPT2Model(
    (wte): Embedding(50258, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (t5): T5ForConditionalGeneration(
    (shared): Embedding(32128, 512)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 512)
      (block): ModuleList(
        (0): T5Block(
  

In [39]:
import time
import tqdm
from text2sql_decoding_utils import decode_sqls
from spider_metric.evaluator import EvaluateTool

dev_filepath = "../data/resdsql_pre/preprocessed_dataset_test.json"
original_dev_filepath = "../data/split/spider_test.json"

batch_size = 1
max_input_length = 43
gpt2_model = 'gpt2'
t5_model = 't5-small'
db_path = "../spider_data/database"
mode = "eval"

In [40]:
start_time = time.time()

# initialize tokenizer
gpt2_tokenizer = GPT2Tokenizer.from_pretrained(gpt2_model)
t5_tokenizer = T5Tokenizer.from_pretrained(t5_model)

gpt2_tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# if isinstance(tokenizer, T5TokenizerFast):
#     tokenizer.add_tokens([AddedToken(" <="), AddedToken(" <")])

dev_dataset = Text2SQLDataset(
            dir_ = dev_filepath,
            mode = mode)

dev_dataloder = DataLoader(
        dev_dataset, 
        batch_size = batch_size, 
        shuffle = False,
        collate_fn = lambda x: x,
        drop_last = False
    )

# initialize model

model2.eval()
predict_sqls = []
# for batch in tqdm(dev_dataloder):
for idx, batch in enumerate(dev_dataloder):
    batch_inputs = [data[0] for data in batch]
    batch_db_ids = [data[1] for data in batch]
    batch_tc_original = [data[2] for data in batch]

    tokenized_inputs = gpt2_tokenizer(batch_inputs,
                                      add_special_tokens=True,
                                      padding="max_length", #True,
                                      max_length=max_input_length,
                                      #pad_to_max_length=True,
                                      return_tensors='pt',
                                      truncation=True)

    encoder_input_ids = tokenized_inputs["input_ids"].to(device)
    encoder_input_attention_mask = tokenized_inputs["attention_mask"].to(device)
    
    print(f"encoder_input_ids - {encoder_input_ids.size()}")
    print(f"encoder_input_attention_mask - {encoder_input_attention_mask.size()}")

    with torch.no_grad():
        model_outputs = model2.predict(encoder_input_ids, encoder_input_attention_mask,
                                       batch_size, t5_tokenizer=t5_tokenizer)

        model_outputs = model_outputs.view(batch_size, 1, model_outputs.shape[1])
        
        predict_sqls += decode_sqls(
                                    db_path, 
                                    model_outputs, 
                                    batch_db_ids, 
                                    batch_inputs, 
                                    t5_tokenizer, 
                                    batch_tc_original
                                    )
    break


new_dir = "/".join(output.split("/")[:-1]).strip()
if new_dir != "":
    os.makedirs(new_dir, exist_ok = True)

# save results
with open(output, "w", encoding = 'utf-8') as f:
    for pred in predict_sqls:
        f.write(pred + "\n")

end_time = time.time()
print("Text-to-SQL inference spends {}s.".format(end_time-start_time))

if mode == "eval":
    # initialize evaluator
    evaluator = EvaluateTool()
    evaluator.register_golds(original_dev_filepath, db_path)
    spider_metric_result = evaluator.evaluate(predict_sqls)
    print('exact_match score: {}'.format(spider_metric_result["exact_match"]))
    print('exec score: {}'.format(spider_metric_result["exec"]))
    
    em_res = spider_metric_result["exact_match"]
    ex_res = spider_metric_result["exec"]
#     return spider_metric_result["exact_match"], spider_metric_result["exec"]

encoder_input_ids - torch.Size([1, 43])
encoder_input_attention_mask - torch.Size([1, 43])
gpt2_output - torch.Size([1, 43, 768])
gpt2_output linear - torch.Size([1, 43, 512])
t5_input - torch.Size([1, 1, 43, 512]) - tensor([[[[-0.0062,  0.0012,  0.1066,  ...,  0.0302,  0.1079, -0.1081],
          [-0.0788, -0.0619, -0.1048,  ...,  0.0212,  0.1037, -0.1099],
          [-0.0513,  0.1958,  0.0470,  ...,  0.0699,  0.2082,  0.2164],
          ...,
          [ 0.0589,  0.1992, -0.0373,  ...,  0.0252,  0.1220, -0.0641],
          [ 0.0616,  0.1989, -0.0402,  ...,  0.0280,  0.1228, -0.0624],
          [ 0.0636,  0.1986, -0.0399,  ...,  0.0273,  0.1219, -0.0626]]]])
decoder_input_ids - torch.Size([1, 1])
decoder_attention_mask - torch.Size([1, 1])
initial decoder_input_ids - tensor([[0]])
t5_outputs logits - torch.Size([1, 1, 32128])
next_token_logits - torch.Size([1, 32128])
pred decoder_input_ids - tensor([[   0, 1738]])
t5_outputs logits - torch.Size([1, 2, 32128])
next_token_logits - torch

RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
model_outputs, model_outputs.shape

In [ ]:
batch_inputs, batch_db_ids, batch_tc_original